In [46]:
from datasets import load_dataset
import spacy # Better than nltk
from spacy.lang.fr.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
#from gensim import corpora
#from gensim.models import LsiModel
#from gensim.models.coherencemodel import CoherenceModel
#import matplotlib.pyplot as plt

In [2]:
dataset = load_dataset('mlsum', 'fr')

Reusing dataset mlsum (C:\Users\maxim\.cache\huggingface\datasets\mlsum\fr\1.0.0\77f23eb185781f439927ac2569ab1da1083195d8b2dab2b2f6bbe52feb600688)


  0%|          | 0/3 [00:00<?, ?it/s]

In [44]:
article = dataset['train']['text'][4]
summary = dataset['train']['summary'][4]
print(article, summary, sep='\n')

Cinq personnes sont mortes, et treize autres ont été blessées à Nîmes, dans le Gard, dans un incendie qui s'est déclenché vendredi 1er janvier au petit matin. Le feu, dont on ignore l'origine pour l'instant, a pris au sixième et dernier étage d'un immeuble. "A l'arrivée des pompiers, trois personnes étaient décédées dans un appartement, et deux autres dans un appartement voisin par intoxication", a expliqué, sur i-Télé, le directeur de cabinet du préfet du Gard. On dénombre également "treize blessés, dont trois graves. Une personne dans le coma a été transférée à Marseille", a-t-il ajouté. Les secours ont été prévenus vers 5 heures du matin, mais "l'incendie avait déjà bien démarré", a-t-il expliqué. France Info précise que les victimes sont trois adultes et deux enfants. "L'origine de l'incendie est indéterminée mais a priori accidentelle", a déclaré le procureur adjoint de la République de Nîmes, cité par Europe 1.
Cinq personnes sont mortes, et treize autres ont été blessées à Nîmes

In [54]:
nlp = spacy.load("fr_core_news_sm")
doc = nlp(article)

sentences = []
cur_sentence = []
for sent in doc.sents:
    for token in sent:
        if not token.text.lower() in STOP_WORDS and not token.is_punct:
            #print(token.text, token.lemma_)
            cur_sentence.append(token.lemma_)
    sentences.append(cur_sentence)
    cur_sentence = []

for s in sentences:
    print(s)
    print()

vectorizer = TfidfVectorizer(ngram_range=(1, 1))
X = vectorizer.fit_transform([' '.join(sent) for sent in sentences])
print(X)

['personne', 'mourir', 'être', 'blesser', 'Nîmes', 'Gard', 'incendie', 'déclencher', 'vendredi', 'premier', 'janvier', 'petit', 'matin']

['feu', 'ignore', 'origine', 'instant', 'prendre', 'dernier', 'étage', 'immeuble']

['arrivée', 'pompier', 'personne', 'décéder', 'appartement', 'appartement', 'voisin', 'intoxication', 'expliquer', 'Télé', 'directeur', 'cabinet', 'préfet', 'Gard']

['dénombre', 'blessé', 'grave']

['coma', 'être', 'transférer', 'Marseille', '-t', 'il', 'ajouté']

['secours', 'être', 'prévenir', '5', 'heure', 'matin', 'incendie', 'bien', 'démarrer', '-t', 'il', 'expliquer']

['France', 'Info', 'précise', 'victime', 'adulte', 'enfant']

['origine', 'incendie', 'indéterminer', 'priori', 'accidentel', 'déclarer', 'procureur', 'adjoint', 'république', 'nîme', 'citer', 'Europe', '1']

  (0, 37)	0.25094924707839145
  (0, 43)	0.29943422171888545
  (0, 35)	0.29943422171888545
  (0, 45)	0.29943422171888545
  (0, 56)	0.29943422171888545
  (0, 15)	0.29943422171888545
  (0, 30)	